In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [6]:
df= pd.concat([df1,df2,df3,df4])

In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,J A N U A R Y,5.0,Hacksaw,Leone Films / Midnight Releasing,"Anthony Leone (director/screenplay); Amy Cay, ...",[3],NaN
2,J A N U A R Y,12.0,Dr. Bird's Advice for Sad Poets,Relativity Media / Ketchup Entertainment,Yaniv Raz (director/screenplay); Lucas Jade Zu...,[4],NaN
3,J A N U A R Y,13.0,The White Tiger,Netflix / ARRAY / Purple Pebble Pictures,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
4,J A N U A R Y,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[5],NaN
...,...,...,...,...,...,...,...
93,D E C E M B E R,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[277]
94,D E C E M B E R,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[278]
95,D E C E M B E R,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[279]
96,D E C E M B E R,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[280]


In [8]:
df_2021 = df[['Title','Cast and crew']]

In [9]:
!pip install tmdbv3api

In [10]:
from tmdbv3api import TMDb
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()

tmdb = TMDb()
tmdb.api_key = os.getenv('API_KEY')

In [11]:
from tmdbv3api import Movie
import numpy as np
import requests

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    try:
        result = tmdb_movie.search(x)
        if not result:
            return np.NaN
        else:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb_movie.api_key))
            data_json = response.json()
            if 'genres' in data_json and data_json['genres']:
                genre_str = " "
                for i in range(len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
    except Exception as e:
        print(f"Error processing title '{x}': {e}")
        return np.NaN

In [12]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

Error processing title 'Wish Dragon': getattr(): attribute name must be string
Error processing title 'Dating and New York': getattr(): attribute name must be string


<ipython-input-12-252c1cda1e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [13]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War
1,Hacksaw,"Anthony Leone (director/screenplay); Amy Cay, ...",Adventure Family TV Movie Western
2,Dr. Bird's Advice for Sad Poets,Yaniv Raz (director/screenplay); Lucas Jade Zu...,Comedy Drama
3,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama
4,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Romance
...,...,...,...
93,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War
94,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance
95,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama
96,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Science Fiction Mystery


In [14]:
df_2021['genres'].isna().sum()

3

In [15]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-16-059d9cfe3ca6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [17]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [18]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-18-fbf9a4980306>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [19]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [20]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-20-597eaf6de001>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [23]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [24]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [25]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,Hacksaw
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,Dr. Bird's Advice for Sad Poets
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,Locked Down
...,...,...,...,...,...,...
93,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth
94,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan
95,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog
96,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Science Fiction Mystery,Memoria


In [26]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [27]:
new_df21.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      8
actor_3_name     26
genres            3
movie_title       1
comb             28
dtype: int64

In [28]:
new_df21 = new_df21.dropna(how='any')

In [29]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [30]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

<ipython-input-30-6e765124d575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title'] = new_df21['movie_title'].str.lower()


In [31]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,hacksaw,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,dr. bird's advice for sad poets,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
...,...,...,...,...,...,...,...
92,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Comedy Adventure Thriller,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
93,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
94,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
95,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [32]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [33]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [34]:
tables = soup.find_all('table',class_='wikitable sortable')

In [35]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [36]:
df= pd.concat([df1,df2,df3,df4])

In [37]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,J A N U A R Y,7.0,The Legend of La Llorona,Saban Films / Ageless Pictures,Patricia Harris Seeley (director); José Prende...,[3],NaN
2,J A N U A R Y,7.0,The Commando,Saban Films / Premiere Entertainment,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,J A N U A R Y,7.0,American Siege,Vertical Entertainment,Edward John Drake (director/screenplay); Timot...,[5],NaN
4,J A N U A R Y,14.0,Scream,Paramount Pictures / Spyglass Media Group / Ra...,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6],NaN
...,...,...,...,...,...,...,...
86,D E C E M B E R,30.0,"Alice, Darling",Lionsgate / Elevation Pictures,Mary Nighy (director); Alanna Francis (screenp...,NaN,[261]
87,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
88,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN
89,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_2022 = df[['Title','Cast and crew']]

In [39]:
!pip install tmdbv3api

In [40]:
from tmdbv3api import TMDb
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()

tmdb = TMDb()
tmdb.api_key = os.getenv('API_KEY')

In [41]:
from tmdbv3api import Movie
import numpy as np
import requests

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    try:
        result = tmdb_movie.search(x)
        if not result:
            return np.NaN
        else:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb_movie.api_key))
            data_json = response.json()
            if 'genres' in data_json and data_json['genres']:
                genre_str = " "
                for i in range(len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
    except Exception as e:
        print(f"Error processing title '{x}': {e}")
        return np.NaN

In [42]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

Error processing title 'Apollo 10 1⁄2: A Space Age Childhood': getattr(): attribute name must be string


<ipython-input-42-4e8e2d4797da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))


In [43]:
df_2022['genres'].isna().sum()

1

In [44]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director); José Prende...,Family Animation Fantasy Horror
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,American Siege,Edward John Drake (director/screenplay); Timot...,Action Adventure Thriller
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Crime Horror Mystery
...,...,...,...
86,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Drama Romance
87,NaN,NaN,Drama Crime
88,NaN,NaN,Drama Crime
89,NaN,NaN,Drama Crime


In [45]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [46]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-46-bde37e357dd2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))


In [47]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [48]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-48-e67b31464e65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [49]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [50]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-50-d3d5c35d26fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [51]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [52]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [54]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [55]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [56]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Crime Horror Mystery,Scream
...,...,...,...,...,...,...
86,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Drama Romance,"Alice, Darling"
87,nan,nan,NaN,NaN,Drama Crime,NaN
88,nan,nan,NaN,NaN,Drama Crime,NaN
89,nan,nan,NaN,NaN,Drama Crime,NaN


In [57]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [58]:
new_df22.isna().sum()

director_name     0
actor_1_name      0
actor_2_name     11
actor_3_name     34
genres            1
movie_title       4
comb             35
dtype: int64

In [59]:
new_df22 = new_df22.dropna(how='any')

In [60]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [61]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

<ipython-input-61-bd701421fe8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df22['movie_title'] = new_df22['movie_title'].str.lower()


In [62]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Thriller,the 355,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,the legend of la llorona,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,american siege,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Crime Horror Mystery,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Derek Drymon, Jennifer Kluska",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Adventure Fantasy,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Derek D...
...,...,...,...,...,...,...,...
82,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
83,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Thriller Crime Horror Mystery,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
84,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
85,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [63]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

In [64]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [65]:
tables = soup.find_all('table',class_='wikitable sortable')

In [66]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [67]:
df=pd.concat([df1,df2,df3,df4])

In [68]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,6,M3GAN,Universal Pictures / Blumhouse Productions / A...,Gerard Johnstone (director); Akela Cooper (scr...,[3]
1,J A N U A R Y,6,The Old Way,Saban Films / Saturn Films,Brett Donowho (director); Carl W. Lucas (scree...,[4]
2,J A N U A R Y,11,The Devil Conspiracy,Samuel Goldwyn Films,Nathan Frankowski (director); Ed Alan (screenp...,[5]
3,J A N U A R Y,13,Plane,Lionsgate / MadRiver Pictures / Di Bonaventura...,Jean-François Richet (director); Charles Cummi...,[6]
4,J A N U A R Y,13,House Party,Warner Bros. Pictures / New Line Cinema,"Calmatic (director); Jamal Olori, Stephen Glov...",[7]
...,...,...,...,...,...,...
100,D E C E M B E R,22,Memory,Ketchup Entertainment / Mubi,Michel Franco (director/screenplay); Jessica C...,[325]
101,D E C E M B E R,25,The Color Purple,Warner Bros. Pictures / Amblin Entertainment /...,"Blitz Bazawule (director), Marcus Gardley (scr...",[326]
102,D E C E M B E R,25,The Boys in the Boat,Metro-Goldwyn-Mayer / Smokehouse Pictures,"George Clooney (director), Mark L. Smith (scre...",[327]
103,D E C E M B E R,25,Ferrari,Neon / STXfilms / Ketchup Entertainment,"Michael Mann (director), Troy Kennedy Martin (...",[328]


In [69]:
df_2023 = df[['Title','Cast and crew']]

In [70]:
!pip install tmdbv3api

In [71]:
from tmdbv3api import TMDb
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()

tmdb = TMDb()
tmdb.api_key = os.getenv('API_KEY')

In [72]:
from tmdbv3api import Movie
import numpy as np
import requests

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    try:
        result = tmdb_movie.search(x)
        if not result:
            return np.NaN
        else:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb_movie.api_key))
            data_json = response.json()
            if 'genres' in data_json and data_json['genres']:
                genre_str = " "
                for i in range(len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
    except Exception as e:
        print(f"Error processing title '{x}': {e}")
        return np.NaN

In [73]:
df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-73-2ed0b991c088>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['genres'] = df_2023['Title'].map(lambda x: get_genre(str(x)))


In [74]:
df_2023

,Title,Cast and crew,genres
0,M3GAN,Gerard Johnstone (director); Akela Cooper (scr...,Science Fiction Horror
1,The Old Way,Brett Donowho (director); Carl W. Lucas (scree...,Western Drama
2,The Devil Conspiracy,Nathan Frankowski (director); Ed Alan (screenp...,Horror Fantasy Science Fiction Thriller
3,Plane,Jean-François Richet (director); Charles Cummi...,Action
4,House Party,"Calmatic (director); Jamal Olori, Stephen Glov...",Comedy
...,...,...,...
100,Memory,Michel Franco (director/screenplay); Jessica C...,Action Thriller Crime
101,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",NaN
102,The Boys in the Boat,"George Clooney (director), Mark L. Smith (scre...",Drama History
103,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",History Drama


In [75]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [76]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-76-6179b260bd79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))


In [77]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [78]:
df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-78-07af9e2bc7c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['actor_1_name'] = df_2023['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [79]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [80]:
df_2023['actor_2_name'] = df_2023['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [81]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [82]:
df_2023['actor_3_name'] = df_2023['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [83]:
df_2023 = df_2023.rename(columns={'Title':'movie_title'})

In [84]:
new_df23 = df_2023.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [85]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,M3GAN
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN,Western Drama,The Old Way
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action,Plane
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party
...,...,...,...,...,...,...
100,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory
101,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,NaN,The Color Purple
102,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat
103,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari


In [86]:
new_df23['comb'] = new_df23['actor_1_name'] + ' ' + new_df23['actor_2_name'] + ' '+ new_df23['actor_3_name'] + ' '+ new_df23['director_name'] +' ' + new_df23['genres']

In [87]:
new_df23.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      3
actor_3_name     20
genres            2
movie_title       0
comb             22
dtype: int64

In [88]:
new_df23=new_df23.dropna(how='any')

In [89]:
new_df23.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [90]:
new_df23

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,M3GAN,Allison Williams Violet McGraw Amie Donald Ger...
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action,Plane,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party,Tosin Cole Jacob Latimore Karen Obilom Calmati...
5,John Hyams,Gideon Adlon,Bethlehem Million,Marc Menchaca,Horror Thriller Mystery,Sick,Gideon Adlon Bethlehem Million Marc Menchaca J...
...,...,...,...,...,...,...,...
99,Matthew Brown,Anthony Hopkins,Matthew Goode,Liv Lisa Fries,Drama,Freud's Last Session,Anthony Hopkins Matthew Goode Liv Lisa Fries M...
100,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory,Jessica Chastain Peter Sarsgaard Merritt Wever...
102,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat,Callum Turner Joel Edgerton Peter Guinness Geo...
103,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari,Adam Driver Penelope Cruz Shailene Woodley Mic...


In [91]:
my_df=pd.concat([new_df21,new_df22,new_df23])

In [92]:
my_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Adventure Family TV Movie Western,hacksaw,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,dr. bird's advice for sad poets,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
...,...,...,...,...,...,...,...
99,Matthew Brown,Anthony Hopkins,Matthew Goode,Liv Lisa Fries,Drama,Freud's Last Session,Anthony Hopkins Matthew Goode Liv Lisa Fries M...
100,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory,Jessica Chastain Peter Sarsgaard Merritt Wever...
102,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat,Callum Turner Joel Edgerton Peter Guinness Geo...
103,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari,Adam Driver Penelope Cruz Shailene Woodley Mic...


In [93]:
old_df=pd.read_csv('updated_data.csv')

In [94]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Joachim Rønning Espen Sandberg,Johnny Depp,Javier Bardem,Geoffrey Rush,Adventure Action Fantasy Comedy,pirates of the caribbean: dead men tell no tales,Johnny Depp Javier Bardem Geoffrey Rush Joachi...
1,Zack Snyder,Ben Affleck,Henry Cavill,Gal Gadot,Action Adventure Fantasy Sci-Fi,justice league,Ben Affleck Henry Cavill Gal Gadot Zack Snyder...
2,Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Action Adventure Fantasy Sci-Fi,thor: ragnarok,Chris Hemsworth Tom Hiddleston Cate Blanchett ...
3,James Gunn,Chris Pratt,Zoe Saldana,Dave Bautista,Action Adventure Comedy Sci-Fi,guardians of the galaxy vol. 2,Chris Pratt Zoe Saldana Dave Bautista James Gu...
4,Sean McNamara,Pierce Brosnan,William Hurt,Benjamin Walker,Fantasy Action Adventure,the king's daughter,Pierce Brosnan William Hurt Benjamin Walker Se...
...,...,...,...,...,...,...,...
6090,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
6091,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Family Action Fantasy Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6092,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6093,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [95]:
last_df=pd.concat([old_df,my_df])

In [96]:
last_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Joachim Rønning Espen Sandberg,Johnny Depp,Javier Bardem,Geoffrey Rush,Adventure Action Fantasy Comedy,pirates of the caribbean: dead men tell no tales,Johnny Depp Javier Bardem Geoffrey Rush Joachi...
1,Zack Snyder,Ben Affleck,Henry Cavill,Gal Gadot,Action Adventure Fantasy Sci-Fi,justice league,Ben Affleck Henry Cavill Gal Gadot Zack Snyder...
2,Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Action Adventure Fantasy Sci-Fi,thor: ragnarok,Chris Hemsworth Tom Hiddleston Cate Blanchett ...
3,James Gunn,Chris Pratt,Zoe Saldana,Dave Bautista,Action Adventure Comedy Sci-Fi,guardians of the galaxy vol. 2,Chris Pratt Zoe Saldana Dave Bautista James Gu...
4,Sean McNamara,Pierce Brosnan,William Hurt,Benjamin Walker,Fantasy Action Adventure,the king's daughter,Pierce Brosnan William Hurt Benjamin Walker Se...
...,...,...,...,...,...,...,...
99,Matthew Brown,Anthony Hopkins,Matthew Goode,Liv Lisa Fries,Drama,Freud's Last Session,Anthony Hopkins Matthew Goode Liv Lisa Fries M...
100,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory,Jessica Chastain Peter Sarsgaard Merritt Wever...
102,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat,Callum Turner Joel Edgerton Peter Guinness Geo...
103,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari,Adam Driver Penelope Cruz Shailene Woodley Mic...


In [97]:
last_df.to_csv('main_data.csv',index=False)